<div style="display:block">
    <div style="width: 20%; display: inline-block; text-align: left;">
        <div class="crop">
            <img src="https://image4.owler.com/logo/rapido_owler_20190125_062402_original.png" style="height:75px; margin-left:0px" />
        </div>
    </div>
    <div style="width: 59%; display: inline-block">
            <h1  style="text-align: center">Pricing Affluence CR Setup</h1><br>
        <div style="width: 90%; text-align: center; display: inline-block;"><i>By:</i> <strong>Manoj Kumar</strong> </div>
    </div>
    <div style="width: 20%; text-align: right; display: inline-block;">
        <div style="width: 100%; text-align: left; display: inline-block;">
            <i>Created: </i>
            <time datetime="Enter Date" pubdate>22nd Aug, 2023</time>
        </div>
    </div>
</div>

___
___

<div class="alert alert-block alert-danger">

# Configs
</div>

<div class="alert alert-block alert-warning">
    
### Enter Configs To Run The Notebook
Comment or Edit the changes in Similar Format
</div>

In [1]:
CITY = 'Bangalore'

In [2]:
START_DATE = '20230821'                    
END_DATE = '20230910'

In [3]:
X_PERCENT_TOP = 100

<div class="alert alert-block alert-danger">

</div>

___

<div class="alert alert-block alert-info">

# Importing Packages
</div>

___

In [4]:
import pandas as pd
import numpy as np
import h3
from datetime import datetime
from pyhive import presto
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
from sklearn.metrics.pairwise import cosine_similarity
from numpy import array, reshape
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [5]:
pd.set_option('display.max_columns', 600)
pd.set_option('display.max_rows', 600)

In [6]:
conn = presto.connect(
    host='presto-gateway.serving.data.plectrum.dev',
    port=443,
    protocol='https',
    catalog='hive',
    username='manoj.ravirajan@rapido.bike')

___

<div class="alert alert-block alert-info">

# Download Data
</div>

___

In [7]:
CITY = 'Bangalore'

In [8]:
START_DATE = '20230821'                    
END_DATE = '20230910'

___

In [9]:
city_cluster_hex_data = pd.read_csv('/Users/rapido/local-datasets/affluence/experiment/experiment_documents/city_cluster_hex.csv')
selected_cluster_hex_data = pd.read_csv('/Users/rapido/local-datasets/affluence/experiment/experiment_documents/selected_cluster_hex.csv')

In [10]:
city_cluster_hex_data.head(2)

,hex_id,cluster
0,8860145229fffff,South Bidadi
1,8860145083fffff,South Bidadi


In [11]:
city_cluster_hex_data.describe()

,hex_id,cluster
count,3113,3113
unique,3113,229
top,8860145229fffff,Bangalore Airport
freq,1,280


In [12]:
city_cluster_hex_data.cluster.nunique()

229

In [13]:
selected_cluster_hex_data.head(2)

,pickup_cluster,pickup_hex8
0,BGS Kengeri,8860145ad1fffff
1,Bagalur,8861892db5fffff


In [14]:
selected_cluster_hex_data.describe()

,pickup_cluster,pickup_hex8
count,36,36
unique,29,36
top,Horamavu,8860145ad1fffff
freq,4,1


In [15]:
selected_cluster_hex_data.count()

pickup_cluster    36
pickup_hex8       36
dtype: int64

In [16]:
selected_cluster_hex_data.pickup_cluster.nunique()

29

___

<div class="alert alert-block alert-info">

# Work around
</div>

In [17]:
city_cluster_hex_data.count()

hex_id     3113
cluster    3113
dtype: int64

In [18]:
selected_cluster_hex_data.count()

pickup_cluster    36
pickup_hex8       36
dtype: int64

In [19]:
city_cluster_df = pd.merge(city_cluster_hex_data, \
                           selected_cluster_hex_data, \
                           how = 'left', \
                           left_on = ['cluster'], \
                           right_on = ['pickup_cluster']
                          )
city_cluster_df.head()

,hex_id,cluster,pickup_cluster,pickup_hex8
0,8860145229fffff,South Bidadi,NaN,NaN
1,8860145083fffff,South Bidadi,NaN,NaN
2,8860144825fffff,Nelamangala,NaN,NaN
3,8860169411fffff,Hesarghatta,NaN,NaN
4,8861892e9bfffff,Metro - MG Road,NaN,NaN


In [20]:
city_cluster_df = city_cluster_df[city_cluster_df['cluster'] != city_cluster_df['pickup_cluster']]

In [21]:
selected_cluster_hex_data.pickup_cluster.unique()

array(['BGS Kengeri', 'Bagalur', 'Banaswadi',
       'Bommasandra Industrial Area', 'Domlur',
       'Electronic City Phase 2 ', 'HBR Layout', 'Harlur', 'Horamavu',
       'K R Puram', 'Kurudusonnehalli', 'Madiwala Lake', 'Majestic',
       'Marathahalli Village', 'Mathikere', 'Mathikere Lake',
       'Metro - Lalbagh', 'Mysore rd', 'Nagavara', 'Neelasandra',
       'R T Nagar', 'Sahakara Nagar', 'Thanisandra', 'Trinity',
       'Vijayanagar', 'Whitefield', 'Yelahanka Traffic Police Station',
       'Yelanhanka', 'Yeshwanthpur - Mathikere'], dtype=object)

In [22]:
city_cluster_df[city_cluster_df['cluster'].isin(['BGS Kengeri', 'Bagalur', 'Banaswadi', \
                                                 'Bommasandra Industrial Area', 'Domlur', \
                                                 'Electronic City Phase 2 ', 'HBR Layout', 'Harlur', 'Horamavu', \
                                                 'K R Puram', 'Kurudusonnehalli', 'Madiwala Lake', 'Majestic', \
                                                 'Marathahalli Village', 'Mathikere', 'Mathikere Lake', \
                                                 'Metro - Lalbagh', 'Mysore rd', 'Nagavara', 'Neelasandra', \
                                                 'R T Nagar', 'Sahakara Nagar', 'Thanisandra', 'Trinity', \
                                                 'Vijayanagar', 'Whitefield', 'Yelahanka Traffic Police Station', \
                                                 'Yelanhanka', 'Yeshwanthpur - Mathikere'])]

,hex_id,cluster,pickup_cluster,pickup_hex8


In [23]:
# city_cluster details after excluding 29 experiment clusters
city_cluster_df = city_cluster_df[['cluster']]
city_cluster_df = city_cluster_df.drop_duplicates()
city_cluster_df.head(5)

,cluster
0,South Bidadi
2,Nelamangala
3,Hesarghatta
4,Metro - MG Road
5,Doddaballapura


In [24]:
city_cluster_df.count()

cluster    200
dtype: int64

In [25]:
## double validation
city_cluster_df[city_cluster_df['cluster'].isin(['BGS Kengeri', 'Bagalur', 'Banaswadi', \
                                                 'Bommasandra Industrial Area', 'Domlur', \
                                                 'Electronic City Phase 2 ', 'HBR Layout', 'Harlur', 'Horamavu', \
                                                 'K R Puram', 'Kurudusonnehalli', 'Madiwala Lake', 'Majestic', \
                                                 'Marathahalli Village', 'Mathikere', 'Mathikere Lake', \
                                                 'Metro - Lalbagh', 'Mysore rd', 'Nagavara', 'Neelasandra', \
                                                 'R T Nagar', 'Sahakara Nagar', 'Thanisandra', 'Trinity', \
                                                 'Vijayanagar', 'Whitefield', 'Yelahanka Traffic Police Station', \
                                                 'Yelanhanka', 'Yeshwanthpur - Mathikere'])]

,cluster


In [26]:
# city_cluster details after excluding 29 experiment clusters

city_cluster_df.head(5)

,cluster
0,South Bidadi
2,Nelamangala
3,Hesarghatta
4,Metro - MG Road
5,Doddaballapura


In [27]:
city_cluster_df['mapping'] = 1

In [28]:
df_week = pd.DataFrame({'mapping': [1,1,1,1,1,1,1], 'week_period': ['Monday','Tuesday',
                                                              'Wednesday','Thursday',
                                                              'Friday','Saturday','Sunday']})

df_week

,mapping,week_period
0,1,Monday
1,1,Tuesday
2,1,Wednesday
3,1,Thursday
4,1,Friday
5,1,Saturday
6,1,Sunday


In [29]:
city_cluster_week_df = city_cluster_df.merge(df_week,on='mapping').drop(columns = 'mapping',axis =1)
city_cluster_week_df.shape

(1400, 2)

In [30]:
200*7

1400

In [31]:
city_cluster_week_df.cluster.nunique()

200

In [32]:
city_cluster_week_df.groupby(['cluster']).agg({'week_period' : set})

,week_period
cluster,
AECS Layout Brookefield,"{Friday, Monday, Wednesday, Sunday, Tuesday, S..."
Adugodi,"{Friday, Monday, Wednesday, Sunday, Tuesday, S..."
Adugodi Traffic Station,"{Friday, Monday, Wednesday, Sunday, Tuesday, S..."
Agara Lake,"{Friday, Monday, Wednesday, Sunday, Tuesday, S..."
Akshaynagar,"{Friday, Monday, Wednesday, Sunday, Tuesday, S..."
Akshaynagar Lake,"{Friday, Monday, Wednesday, Sunday, Tuesday, S..."
Anepalya,"{Friday, Monday, Wednesday, Sunday, Tuesday, S..."
Anjanapura,"{Friday, Monday, Wednesday, Sunday, Tuesday, S..."
Arekere,"{Friday, Monday, Wednesday, Sunday, Tuesday, S..."


In [33]:
city_cluster_week_df.rename(columns = {'cluster':'pickup_cluster'}, inplace = True)
city_cluster_week_df['drop_cluster'] = 'ALL'
city_cluster_week_df['start_time'] = '0'
city_cluster_week_df['end_time'] = '2359'

___

<div class="alert alert-block alert-danger">

# Ciruit Breaker Recommendation
</div>

___

In [34]:
city_cluster_week_df = city_cluster_week_df[['pickup_cluster', 'drop_cluster', 'week_period', 'start_time', 'end_time']]

In [35]:
city_cluster_week_df

,pickup_cluster,drop_cluster,week_period,start_time,end_time
0,South Bidadi,ALL,Monday,0,2359
1,South Bidadi,ALL,Tuesday,0,2359
2,South Bidadi,ALL,Wednesday,0,2359
3,South Bidadi,ALL,Thursday,0,2359
4,South Bidadi,ALL,Friday,0,2359
...,...,...,...,...,...
1395,Ulsoor,ALL,Wednesday,0,2359
1396,Ulsoor,ALL,Thursday,0,2359
1397,Ulsoor,ALL,Friday,0,2359
1398,Ulsoor,ALL,Saturday,0,2359


In [36]:
city_cluster_week_df['city'] = CITY

In [37]:
final_rec_v1 = city_cluster_week_df[[ 'city','pickup_cluster', 'drop_cluster', 'week_period', 'start_time','end_time']]

In [38]:
final_rec_v1['pickup_hex8'] = 'ALL'
final_rec_v1['drop_hex8'] = 'ALL'

/var/folders/j7/5rtfb17j30s9g9q790v9nr0h0000gn/T/ipykernel_28250/3500822544.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_rec_v1['pickup_hex8'] = 'ALL'
/var/folders/j7/5rtfb17j30s9g9q790v9nr0h0000gn/T/ipykernel_28250/3500822544.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_rec_v1['drop_hex8'] = 'ALL'


In [39]:
final_rec_v1

,city,pickup_cluster,drop_cluster,week_period,start_time,end_time,pickup_hex8,drop_hex8
0,Bangalore,South Bidadi,ALL,Monday,0,2359,ALL,ALL
1,Bangalore,South Bidadi,ALL,Tuesday,0,2359,ALL,ALL
2,Bangalore,South Bidadi,ALL,Wednesday,0,2359,ALL,ALL
3,Bangalore,South Bidadi,ALL,Thursday,0,2359,ALL,ALL
4,Bangalore,South Bidadi,ALL,Friday,0,2359,ALL,ALL
...,...,...,...,...,...,...,...,...
1395,Bangalore,Ulsoor,ALL,Wednesday,0,2359,ALL,ALL
1396,Bangalore,Ulsoor,ALL,Thursday,0,2359,ALL,ALL
1397,Bangalore,Ulsoor,ALL,Friday,0,2359,ALL,ALL
1398,Bangalore,Ulsoor,ALL,Saturday,0,2359,ALL,ALL


<div class="alert alert-block alert-warning">

</div>

<div class="alert alert-block alert-warning">

</div>

In [40]:
final_recommendation_v1 = final_rec_v1[['city','pickup_cluster', 'drop_cluster', 'week_period', 
                                     'start_time','end_time','pickup_hex8','drop_hex8']]

In [41]:
final_recommendation_v1 = final_recommendation_v1.reset_index(drop=True)
final_recommendation_v1

,city,pickup_cluster,drop_cluster,week_period,start_time,end_time,pickup_hex8,drop_hex8
0,Bangalore,South Bidadi,ALL,Monday,0,2359,ALL,ALL
1,Bangalore,South Bidadi,ALL,Tuesday,0,2359,ALL,ALL
2,Bangalore,South Bidadi,ALL,Wednesday,0,2359,ALL,ALL
3,Bangalore,South Bidadi,ALL,Thursday,0,2359,ALL,ALL
4,Bangalore,South Bidadi,ALL,Friday,0,2359,ALL,ALL
...,...,...,...,...,...,...,...,...
1395,Bangalore,Ulsoor,ALL,Wednesday,0,2359,ALL,ALL
1396,Bangalore,Ulsoor,ALL,Thursday,0,2359,ALL,ALL
1397,Bangalore,Ulsoor,ALL,Friday,0,2359,ALL,ALL
1398,Bangalore,Ulsoor,ALL,Saturday,0,2359,ALL,ALL


In [42]:
rec_name_v1 = CITY[:3]+'_'+START_DATE[-2:]+'_'+END_DATE[-2:]+'_cr_recommendation_v1.parquet'

In [43]:
rec_name_v1

'Ban_21_10_cr_recommendation_v1.parquet'

In [44]:
final_recommendation_v1.to_parquet(rec_name_v1,index=False)

In [45]:
## Half work done here 

final_recommendation_v1

,city,pickup_cluster,drop_cluster,week_period,start_time,end_time,pickup_hex8,drop_hex8
0,Bangalore,South Bidadi,ALL,Monday,0,2359,ALL,ALL
1,Bangalore,South Bidadi,ALL,Tuesday,0,2359,ALL,ALL
2,Bangalore,South Bidadi,ALL,Wednesday,0,2359,ALL,ALL
3,Bangalore,South Bidadi,ALL,Thursday,0,2359,ALL,ALL
4,Bangalore,South Bidadi,ALL,Friday,0,2359,ALL,ALL
...,...,...,...,...,...,...,...,...
1395,Bangalore,Ulsoor,ALL,Wednesday,0,2359,ALL,ALL
1396,Bangalore,Ulsoor,ALL,Thursday,0,2359,ALL,ALL
1397,Bangalore,Ulsoor,ALL,Friday,0,2359,ALL,ALL
1398,Bangalore,Ulsoor,ALL,Saturday,0,2359,ALL,ALL


<div class="alert alert-block alert-warning">

</div>

___

<div class="alert alert-block alert-warning">

</div>

In [46]:
## Second half

In [47]:
city_cluster_df1 = pd.merge(city_cluster_hex_data['cluster'], \
                           selected_cluster_hex_data['pickup_cluster'], \
                           how = 'inner', \
                           left_on = ['cluster'], \
                           right_on = ['pickup_cluster']
                          )
city_cluster_df1 = city_cluster_df1.drop_duplicates()
city_cluster_df1.head()

,cluster,pickup_cluster
0,Neelasandra,Neelasandra
3,Nagavara,Nagavara
11,Kurudusonnehalli,Kurudusonnehalli
39,Bagalur,Bagalur
92,Yeshwanthpur - Mathikere,Yeshwanthpur - Mathikere


In [48]:
city_cluster_df1 = city_cluster_df1[['pickup_cluster']]
city_cluster_df1.pickup_cluster.nunique()

29

In [49]:
city_cluster_29 = pd.merge(city_cluster_hex_data, \
                           city_cluster_df1, \
                           how = 'inner', \
                           left_on = ['cluster'], \
                           right_on = ['pickup_cluster']
                          )
city_cluster_29 = city_cluster_29[['pickup_cluster', 'hex_id']]
city_cluster_29.head()

,pickup_cluster,hex_id
0,Neelasandra,88618925e7fffff
1,Neelasandra,88618925a1fffff
2,Neelasandra,88618925a9fffff
3,Nagavara,8861892c81fffff
4,Nagavara,8861892cc3fffff


In [50]:
city_cluster_hex_data.cluster.nunique()

229

In [51]:
city_cluster_hex_data.hex_id.nunique()

3113

In [52]:
city_cluster_29.pickup_cluster.nunique()

29

In [53]:
city_cluster_29.hex_id.nunique()

363

In [54]:
363-36

327

In [55]:
city_cluster_29_flag = pd.merge(city_cluster_29, \
                           selected_cluster_hex_data, \
                           how = 'left', \
                           left_on = ['pickup_cluster', 'hex_id'], \
                           right_on = ['pickup_cluster', 'pickup_hex8']
                          )
city_cluster_29_flag.head()

,pickup_cluster,hex_id,pickup_hex8
0,Neelasandra,88618925e7fffff,NaN
1,Neelasandra,88618925a1fffff,NaN
2,Neelasandra,88618925a9fffff,88618925a9fffff
3,Nagavara,8861892c81fffff,NaN
4,Nagavara,8861892cc3fffff,NaN


In [56]:
city_cluster_29_flag.pickup_cluster.nunique()

29

In [57]:
city_cluster_29_flag['flag'] = np.where(city_cluster_29_flag['hex_id'] == city_cluster_29_flag['pickup_hex8'], \
                                        'Exclude', \
                                        'Include'
                                       )

In [58]:
city_cluster_29_flag.groupby(['flag']).agg({'hex_id' : 'count'})

,hex_id
flag,
Exclude,36
Include,327


In [59]:
city_cluster_29_flag.groupby(['pickup_cluster']).agg({'hex_id' : 'nunique', 'flag' : set})

,hex_id,flag
pickup_cluster,,
BGS Kengeri,15,"{Include, Exclude}"
Bagalur,53,"{Include, Exclude}"
Banaswadi,14,"{Exclude, Include}"
Bommasandra Industrial Area,22,"{Include, Exclude}"
Domlur,10,"{Include, Exclude}"
Electronic City Phase 2,9,"{Include, Exclude}"
HBR Layout,10,"{Exclude, Include}"
Harlur,4,"{Exclude, Include}"
Horamavu,59,"{Include, Exclude}"


In [60]:
city_cluster_without_36_hex = city_cluster_29_flag[city_cluster_29_flag['flag'].isin(['Include'])]
city_cluster_without_36_hex.count()

pickup_cluster    327
hex_id            327
pickup_hex8         0
flag              327
dtype: int64

In [61]:
city_cluster_without_36_hex.pickup_cluster.nunique()

23

In [62]:
29-6

23

In [63]:
# validation
city_cluster_without_36_hex[city_cluster_without_36_hex['hex_id'] \
                            .isin(['88618921d3fffff', '8861892665fffff', '8861892c97fffff',
                                   '88618920b3fffff', '8861892ed9fffff', '8861892c11fffff',
                                   '8861892c55fffff', '886189246dfffff', '8861892ee5fffff',
                                   '88618925bbfffff', '8861892431fffff', '8860145b43fffff',
                                   '88618925a5fffff', '886014594bfffff', '8861892ca3fffff',
                                   '8860145a05fffff', '8860145943fffff', '886014584dfffff',
                                   '88618925a9fffff', '8861892c0bfffff', '8860145ad1fffff',
                                   '886016966bfffff', '8861892645fffff', '8861892ccbfffff',
                                   '8861892f13fffff', '88618921e3fffff', '8861892e2bfffff',
                                   '8860145b67fffff', '8861892db5fffff', '8861892db7fffff',
                                   '8861892c95fffff', '8861892eb7fffff', '8860145b53fffff',
                                   '8860145b55fffff', '8861892e85fffff', '8860169663fffff']
)]

,pickup_cluster,hex_id,pickup_hex8,flag


In [64]:
city_cluster_without_36_hex = city_cluster_without_36_hex[['pickup_cluster', 'hex_id']]
city_cluster_without_36_hex.rename(columns = {'hex_id' : 'pickup_hex8'}, inplace = True)
city_cluster_without_36_hex.head()

,pickup_cluster,pickup_hex8
0,Neelasandra,88618925e7fffff
1,Neelasandra,88618925a1fffff
3,Nagavara,8861892c81fffff
4,Nagavara,8861892cc3fffff
5,Nagavara,8861892c8dfffff


In [65]:
city_cluster_without_36_hex.count()

pickup_cluster    327
pickup_hex8       327
dtype: int64

In [66]:
city_cluster_without_36_hex['mapping'] = 1

In [67]:
df_week = pd.DataFrame({'mapping': [1,1,1,1,1,1,1], 'week_period': ['Monday','Tuesday',
                                                              'Wednesday','Thursday',
                                                              'Friday','Saturday','Sunday']})

df_week

,mapping,week_period
0,1,Monday
1,1,Tuesday
2,1,Wednesday
3,1,Thursday
4,1,Friday
5,1,Saturday
6,1,Sunday


In [68]:
city_cluster_without_36_hex = city_cluster_without_36_hex.merge(df_week,on='mapping').drop(columns = 'mapping',axis =1)
city_cluster_without_36_hex.shape

(2289, 3)

In [69]:
327*7

2289

In [70]:
city_cluster_without_36_hex.head()

,pickup_cluster,pickup_hex8,week_period
0,Neelasandra,88618925e7fffff,Monday
1,Neelasandra,88618925e7fffff,Tuesday
2,Neelasandra,88618925e7fffff,Wednesday
3,Neelasandra,88618925e7fffff,Thursday
4,Neelasandra,88618925e7fffff,Friday


In [71]:
city_cluster_without_36_hex['city'] = 'Bangalore'
city_cluster_without_36_hex['drop_cluster'] = 'ALL'
city_cluster_without_36_hex['drop_hex8'] = 'ALL'
city_cluster_without_36_hex['start_time'] = '0'
city_cluster_without_36_hex['end_time'] = '2359'

In [72]:
city_cluster_without_36_hex = city_cluster_without_36_hex[['city', 'pickup_cluster', 'drop_cluster', 'week_period', 'start_time', 'end_time', 'pickup_hex8', 'drop_hex8']]

In [73]:
city_cluster_without_36_hex

,city,pickup_cluster,drop_cluster,week_period,start_time,end_time,pickup_hex8,drop_hex8
0,Bangalore,Neelasandra,ALL,Monday,0,2359,88618925e7fffff,ALL
1,Bangalore,Neelasandra,ALL,Tuesday,0,2359,88618925e7fffff,ALL
2,Bangalore,Neelasandra,ALL,Wednesday,0,2359,88618925e7fffff,ALL
3,Bangalore,Neelasandra,ALL,Thursday,0,2359,88618925e7fffff,ALL
4,Bangalore,Neelasandra,ALL,Friday,0,2359,88618925e7fffff,ALL
...,...,...,...,...,...,...,...,...
2284,Bangalore,Madiwala Lake,ALL,Wednesday,0,2359,8861892435fffff,ALL
2285,Bangalore,Madiwala Lake,ALL,Thursday,0,2359,8861892435fffff,ALL
2286,Bangalore,Madiwala Lake,ALL,Friday,0,2359,8861892435fffff,ALL
2287,Bangalore,Madiwala Lake,ALL,Saturday,0,2359,8861892435fffff,ALL


In [74]:
city_cluster_without_36_hex.pickup_cluster.nunique()

23

In [75]:
final_rec_v2 = city_cluster_without_36_hex

In [76]:
final_recommendation_v2 = final_rec_v2[['city','pickup_cluster', 'drop_cluster', 'week_period', 
                                     'start_time','end_time','pickup_hex8','drop_hex8']]

In [77]:
final_recommendation_v1 = final_recommendation_v2.reset_index(drop=True)
final_recommendation_v1

,city,pickup_cluster,drop_cluster,week_period,start_time,end_time,pickup_hex8,drop_hex8
0,Bangalore,Neelasandra,ALL,Monday,0,2359,88618925e7fffff,ALL
1,Bangalore,Neelasandra,ALL,Tuesday,0,2359,88618925e7fffff,ALL
2,Bangalore,Neelasandra,ALL,Wednesday,0,2359,88618925e7fffff,ALL
3,Bangalore,Neelasandra,ALL,Thursday,0,2359,88618925e7fffff,ALL
4,Bangalore,Neelasandra,ALL,Friday,0,2359,88618925e7fffff,ALL
...,...,...,...,...,...,...,...,...
2284,Bangalore,Madiwala Lake,ALL,Wednesday,0,2359,8861892435fffff,ALL
2285,Bangalore,Madiwala Lake,ALL,Thursday,0,2359,8861892435fffff,ALL
2286,Bangalore,Madiwala Lake,ALL,Friday,0,2359,8861892435fffff,ALL
2287,Bangalore,Madiwala Lake,ALL,Saturday,0,2359,8861892435fffff,ALL


In [78]:
rec_name_v2 = CITY[:3]+'_'+START_DATE[-2:]+'_'+END_DATE[-2:]+'_cr_recommendation_v2.parquet'

In [79]:
rec_name_v2

'Ban_21_10_cr_recommendation_v2.parquet'

In [80]:
rec_name_v2

'Ban_21_10_cr_recommendation_v2.parquet'

In [81]:
final_recommendation_v1.to_parquet(rec_name_v2,index=False)

<div class="alert alert-block alert-warning">

</div>

___

___

<div class="alert alert-block alert-warning">

</div>

___

In [82]:
Ban_21_10_cr_recommendation_v1 = pd.read_parquet('Ban_21_10_cr_recommendation_v1.parquet')
Ban_21_10_cr_recommendation_v2 = pd.read_parquet('Ban_21_10_cr_recommendation_v2.parquet')

In [83]:
Ban_21_10_cr_recommendation_v1

,city,pickup_cluster,drop_cluster,week_period,start_time,end_time,pickup_hex8,drop_hex8
0,Bangalore,South Bidadi,ALL,Monday,0,2359,ALL,ALL
1,Bangalore,South Bidadi,ALL,Tuesday,0,2359,ALL,ALL
2,Bangalore,South Bidadi,ALL,Wednesday,0,2359,ALL,ALL
3,Bangalore,South Bidadi,ALL,Thursday,0,2359,ALL,ALL
4,Bangalore,South Bidadi,ALL,Friday,0,2359,ALL,ALL
...,...,...,...,...,...,...,...,...
1395,Bangalore,Ulsoor,ALL,Wednesday,0,2359,ALL,ALL
1396,Bangalore,Ulsoor,ALL,Thursday,0,2359,ALL,ALL
1397,Bangalore,Ulsoor,ALL,Friday,0,2359,ALL,ALL
1398,Bangalore,Ulsoor,ALL,Saturday,0,2359,ALL,ALL


In [84]:
Ban_21_10_cr_recommendation_v2

,city,pickup_cluster,drop_cluster,week_period,start_time,end_time,pickup_hex8,drop_hex8
0,Bangalore,Neelasandra,ALL,Monday,0,2359,88618925e7fffff,ALL
1,Bangalore,Neelasandra,ALL,Tuesday,0,2359,88618925e7fffff,ALL
2,Bangalore,Neelasandra,ALL,Wednesday,0,2359,88618925e7fffff,ALL
3,Bangalore,Neelasandra,ALL,Thursday,0,2359,88618925e7fffff,ALL
4,Bangalore,Neelasandra,ALL,Friday,0,2359,88618925e7fffff,ALL
...,...,...,...,...,...,...,...,...
2284,Bangalore,Madiwala Lake,ALL,Wednesday,0,2359,8861892435fffff,ALL
2285,Bangalore,Madiwala Lake,ALL,Thursday,0,2359,8861892435fffff,ALL
2286,Bangalore,Madiwala Lake,ALL,Friday,0,2359,8861892435fffff,ALL
2287,Bangalore,Madiwala Lake,ALL,Saturday,0,2359,8861892435fffff,ALL


In [85]:
Ban_21_10_cr_recommendation_v1.pickup_cluster.nunique()

200

In [86]:
Ban_21_10_cr_recommendation_v1[Ban_21_10_cr_recommendation_v1['pickup_cluster'].isin(['BGS Kengeri', 'Bagalur', 'Banaswadi', \
                                                 'Bommasandra Industrial Area', 'Domlur', \
                                                 'Electronic City Phase 2 ', 'HBR Layout', 'Harlur', 'Horamavu', \
                                                 'K R Puram', 'Kurudusonnehalli', 'Madiwala Lake', 'Majestic', \
                                                 'Marathahalli Village', 'Mathikere', 'Mathikere Lake', \
                                                 'Metro - Lalbagh', 'Mysore rd', 'Nagavara', 'Neelasandra', \
                                                 'R T Nagar', 'Sahakara Nagar', 'Thanisandra', 'Trinity', \
                                                 'Vijayanagar', 'Whitefield', 'Yelahanka Traffic Police Station', \
                                                 'Yelanhanka', 'Yeshwanthpur - Mathikere'])]

,city,pickup_cluster,drop_cluster,week_period,start_time,end_time,pickup_hex8,drop_hex8


In [87]:
Ban_21_10_cr_recommendation_v2.pickup_cluster.nunique()

23

In [88]:
Ban_21_10_cr_recommendation_v2[Ban_21_10_cr_recommendation_v2['pickup_hex8'] \
                            .isin(['88618921d3fffff', '8861892665fffff', '8861892c97fffff',
                                   '88618920b3fffff', '8861892ed9fffff', '8861892c11fffff',
                                   '8861892c55fffff', '886189246dfffff', '8861892ee5fffff',
                                   '88618925bbfffff', '8861892431fffff', '8860145b43fffff',
                                   '88618925a5fffff', '886014594bfffff', '8861892ca3fffff',
                                   '8860145a05fffff', '8860145943fffff', '886014584dfffff',
                                   '88618925a9fffff', '8861892c0bfffff', '8860145ad1fffff',
                                   '886016966bfffff', '8861892645fffff', '8861892ccbfffff',
                                   '8861892f13fffff', '88618921e3fffff', '8861892e2bfffff',
                                   '8860145b67fffff', '8861892db5fffff', '8861892db7fffff',
                                   '8861892c95fffff', '8861892eb7fffff', '8860145b53fffff',
                                   '8860145b55fffff', '8861892e85fffff', '8860169663fffff']
)]

,city,pickup_cluster,drop_cluster,week_period,start_time,end_time,pickup_hex8,drop_hex8


In [89]:
frames =  [Ban_21_10_cr_recommendation_v1,Ban_21_10_cr_recommendation_v2]
results = pd.concat(frames)

In [90]:
results.to_parquet('/Users/rapido/local-datasets/affluence/experiment/experiment_documents/final_results.parquet',index=False)

### Cluster 200 + 23 + 6
### Hex 200(all) + 23 (327) 